In [2]:
import sys
sys.path.append('..')

In [3]:
from src.data.load_data import get_hackernews_files, load_hackernews_dataframe

In [4]:
files = get_hackernews_files()

In [5]:
path = [x for x in files if x.endswith('HN_2017-01')][0]

In [6]:
df = load_hackernews_dataframe(path)
df.head()

,by,dead,deleted,descendants,id,kids,parent,parts,poll,retrieved_on,score,text,time,title,type,url
0,jfc,NaN,NaN,NaN,13293889,NaN,13291930.0,NaN,NaN,1500612003,NaN,An extraordinary artist. I&#x27;m glad he was ...,1483228806,NaN,comment,NaN
1,greglindahl,NaN,NaN,NaN,13293890,NaN,13292161.0,NaN,NaN,1500612003,NaN,"And for those who are wondering what I meant, ...",1483228834,NaN,comment,NaN
2,imron,NaN,NaN,NaN,13293891,NaN,13293408.0,NaN,NaN,1500612003,NaN,Perhaps they&#x27;re trying to learn from the ...,1483228850,NaN,comment,NaN
3,zzzcpan,NaN,NaN,NaN,13293892,[13293927],13293875.0,NaN,NaN,1500612002,NaN,Russians are the most likely users of the Ukra...,1483228857,NaN,comment,NaN
4,theWatcher37,NaN,NaN,NaN,13293893,"[13293974, 13295659]",13293826.0,NaN,NaN,1500612002,NaN,Crowd funding makes sense however when there i...,1483228879,NaN,comment,NaN


In [7]:
df.count()

by              235752
dead             11727
deleted           7055
descendants      25513
id              242807
kids            103821
parent          208530
parts                5
poll                53
retrieved_on    242807
score            33023
text            204548
time            242807
title            32970
type            242807
url              30994
dtype: int64

In [20]:
comments = df[df['type'] == 'comment']
users = comments.groupby(comments['by']).size()

In [82]:
filtered_users, _ = zip(*users.sort_values(ascending=False).head(100).items())
unique_users = set(filtered_users)

In [83]:
user_df = df[df['by'].map(lambda x: x in unique_users) == True]

In [84]:
from igraph import Graph
graph = Graph()
author_index = set()

In [85]:
import math
from tqdm import tqdm


def add_non_existent_node(author, graph):
    if author not in author_index:
        author_index.add(author)
        graph.add_vertices(author)
    return graph


def extend_graph(df, graph):
    for _, row in tqdm(df.iterrows()):
        author = row['by']
        graph = add_non_existent_node(author, graph)
        parent = row['parent']
        if math.isnan(parent):
            continue
        parent_df = df[df['id'] == int(parent)]
        if len(parent_df) == 0:
            continue
        parent_author = parent_df.iloc[0]['by']
        graph = add_non_existent_node(parent_author, graph)
        graph.add_edges([(author, parent_author)])
    return graph

In [86]:
story_graph = extend_graph(user_df, graph)

19958it [00:12, 1563.14it/s]


In [87]:
story_graph.ecount()



1884

In [89]:
# https://stackoverflow.com/questions/25254151/using-igraph-in-python-for-community-detection-and-writing-community-number-for
dendogram = story_graph.community_edge_betweenness()


In [94]:
print(dendogram)

Dendrogram, 100 elements, 99 merges


In [95]:
type(dendogram)

igraph.clustering.VertexDendrogram

In [96]:
dendogram.optimal_count

45